In [1]:
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
with open("results.pkl", 'rb') as f:
    loaded = pickle.load(f, encoding="bytes") 
    
loaded.keys()

dict_keys([b'Monday', b'Tuesday', b'Friday', b'Wednesday', b'Thursday', b'Sunday', b'Saturday'])

In [3]:
def unravel_values(loaded):
    for day in loaded.keys():
        for tw in loaded[day].keys():
            for mr in loaded[day][tw].keys():
                for hw in loaded[day][tw][mr].keys():
                    for dr in loaded[day][tw][mr][hw].keys():
                        for obj in loaded[day][tw][mr][hw][dr].keys():
                            yield {'day': day.decode("utf-8"),
                               'tw': tw.decode("utf-8").replace("tw=", ""),
                               'mr': float(mr.decode("utf-8").replace("mr=", "")),
                               'hw': float(hw.decode("utf-8").replace("hw=", "")),
                               'dr': float(dr.decode("utf-8").replace("dr=", "")),
                               'attribute': obj.decode("utf-8"),
                               'value': loaded[day][tw][mr][hw][dr][obj]}
                        
gen = unravel_values(loaded)
df = pd.DataFrame(list(gen))
df.head()

,day,tw,mr,hw,dr,attribute,value
0,Monday,"[8, 11]",20.0,20.0,1.0,objective_value,335.535
1,Monday,"[8, 11]",20.0,20.0,1.0,number_of_buses,7
2,Monday,"[8, 11]",20.0,20.0,1.0,cap_list,"[0.0, 11.0, 25.0, 19.0, 24.0, 13.0, 19.0, 14.0..."
3,Monday,"[8, 11]",20.0,20.0,1.0,partition_theo_bus_num,8
4,Monday,"[8, 11]",20.0,20.0,1.0,partition_sim_bus_num,11


In [4]:
df.query("day=='Monday' and tw=='[8, 11]' and mr==20 and hw==20 and dr==1")

,day,tw,mr,hw,dr,attribute,value
0,Monday,"[8, 11]",20.0,20.0,1.0,objective_value,335.535
1,Monday,"[8, 11]",20.0,20.0,1.0,number_of_buses,7
2,Monday,"[8, 11]",20.0,20.0,1.0,cap_list,"[0.0, 11.0, 25.0, 19.0, 24.0, 13.0, 19.0, 14.0..."
3,Monday,"[8, 11]",20.0,20.0,1.0,partition_theo_bus_num,8
4,Monday,"[8, 11]",20.0,20.0,1.0,partition_sim_bus_num,11
5,Monday,"[8, 11]",20.0,20.0,1.0,theo_bus_num,6
6,Monday,"[8, 11]",20.0,20.0,1.0,loop_time,137.016
7,Monday,"[8, 11]",20.0,20.0,1.0,sim_bus_num,8
8,Monday,"[8, 11]",20.0,20.0,1.0,number_list,"[0, 8, 4, 14, 3, 18, 7, 13, 17, 6, 16, 5, 15, ..."
9,Monday,"[8, 11]",20.0,20.0,1.0,route_partition,"{0: [b'R', b'B', b'D', b'R'], 1: [b'R', b'C', ..."


In [5]:
single_value_attributes = ["objective_value", "number_of_buses", "partition_theo_bus_num", 
                           "partition_sim_bus_num", "partition_sim_bus_num", "theo_bus_num",
                           "loop_time", "sim_bus_num", "hourly_cost"]

sdf = df[df.attribute.isin(single_value_attributes)].copy()
sdf.value = sdf.value.astype(float)
sdf.to_csv("single_values.csv")

In [7]:
mdf = df[df.attribute.isin(single_value_attributes)==False].copy()
mdf

,day,tw,mr,hw,dr,attribute,value
2,Monday,"[8, 11]",20.0,20.0,1.0,cap_list,"[0.0, 11.0, 25.0, 19.0, 24.0, 13.0, 19.0, 14.0..."
8,Monday,"[8, 11]",20.0,20.0,1.0,number_list,"[0, 8, 4, 14, 3, 18, 7, 13, 17, 6, 16, 5, 15, ..."
9,Monday,"[8, 11]",20.0,20.0,1.0,route_partition,"{0: [b'R', b'B', b'D', b'R'], 1: [b'R', b'C', ..."
11,Monday,"[8, 11]",20.0,20.0,1.0,geo_route,"[b'R', b'B', b'D', b'R', b'C', b'R', b'A', b'R..."
12,Monday,"[8, 11]",20.0,20.0,1.0,geo_list,"[b'R', b'R', b'R', b'B', b'B', b'D', b'D', b'R..."
...,...,...,...,...,...,...,...
16213,Saturday,"[16, 19]",25.0,15.0,0.8,cap_list,"[0.0, 4.0, 13.0, 4.0, 10.0, 6.0, 11.0, 6.0, 0...."
16219,Saturday,"[16, 19]",25.0,15.0,0.8,number_list,"[0, 8, 10, 20, 9, 18, 7, 17, 19, 6, 4, 2, 16, ..."
16220,Saturday,"[16, 19]",25.0,15.0,0.8,route_partition,"{0: [b'R', b'E', b'D', b'R'], 1: [b'R', b'C', ..."
16222,Saturday,"[16, 19]",25.0,15.0,0.8,geo_route,"[b'R', b'E', b'D', b'R', b'C', b'B', b'A', b'R']"
